In [1]:
import numpy as np
import tensorflow as tf
import sys
sys.path.append('../nnBuilder')
from nnLayer import *
from nnInput import *
from nnTrainer import *
from nnHandler import *
#import time, os, gc
#from IPython.display import clear_output, Image, display, HTML
#from mpl_toolkits.mplot3d import Axes3D
#import shutil
#import matplotlib.pyplot as plt
#%matplotlib inline

In [2]:
data=Layer(type="MNIST",batch=128)
layers=[]
layers.append({"type":"Convolution","pad":"SAME","window":5,"stride":1,"size":32,"relu":True})
layers.append({"type":"Pool","pad":"SAME","window":2,"stride":2,"pool_type":"max"})
layers.append({"type":"Convolution","pad":"SAME","window":5,"stride":1,"size":64,"relu":True})
layers.append({"type":"Pool","pad":"SAME","window":2,"stride":2,"pool_type":"max"})
layers.append({"type":"Relu","size":64})
layers.append({"type":"Linear","size":10,"dropout":True})
network=Layer(type="Network",x=data,layers=layers)
trainer=ClassifierTrainer(network=network,optimizer="adam",finish=True)
sess=SessManager(data,network,trainer)
sess.start()

Extracting /tmp/tensorflow/mnist/input_data\train-images-idx3-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data\train-labels-idx1-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data\t10k-images-idx3-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data\t10k-labels-idx1-ubyte.gz
Starting new session


In [3]:
trainer.train(0,keep_rate=1.)
trainer.eval_error()
for i in range(20):
    trainer.train(100,keep_rate=0.5)
    trainer.train(0,keep_rate=1.)
    trainer.eval_error() #Training error

Error rate: 0.9
Error rate: 0.101563
Error rate: 0.046875
Error rate: 0.0460938
Error rate: 0.0375
Error rate: 0.028125
Error rate: 0.0257812
Error rate: 0.025
Error rate: 0.0242187
Error rate: 0.021875
Error rate: 0.0148437
Error rate: 0.0171875
Error rate: 0.0164062
Error rate: 0.0140625
Error rate: 0.0125
Error rate: 0.0164062
Error rate: 0.0078125
Error rate: 0.009375
Error rate: 0.0101562
Error rate: 0.0109375
Error rate: 0.0117188
